# Participant Exclusion

There are 10 ‘attention check’ trials randomly dispersed in the experiment. On these trials participants are shown an image, but told what to respond (e.g. Click ‘1’ for this question). Participants must get 8 out of 10 attention check trials correct to be included in the analysis. 

In [42]:
import csv
import pandas as pd
import numpy as np
import json

In [43]:
path = '/Users/prachimahableshwarkar/Downloads/Mean Depth Ranking Task_March 27, 2023_09.08.csv'

df = pd.read_csv(path)

In [44]:
catch = {'Q336': 1, 'Q337': 1, 'Q338': 2, 'Q339': 2, 'Q340': 3, 'Q341': 3, 'Q342': 4, 'Q343': 4, 'Q344': 5, 'Q345': 5}

In [45]:
[col for col in df.columns if 'Q' not in col]

['StartDate',
 'EndDate',
 'Status',
 'IPAddress',
 'Progress',
 'Duration (in seconds)',
 'Finished',
 'RecordedDate',
 'ResponseId',
 'RecipientLastName',
 'RecipientFirstName',
 'RecipientEmail',
 'ExternalReference',
 'LocationLatitude',
 'LocationLongitude',
 'DistributionChannel',
 'UserLanguage']

In [46]:
participant_ids = [rid for rid in df.ResponseId.unique() if 'R_' in rid]
len(participant_ids), participant_ids[0]

(50, 'R_ei0emQmbFYFOQuJ')

In [58]:
exclude = []
nans = []
for participant in participant_ids:
    p_df = df.loc[df['ResponseId'] == participant]
    num_correct = 0
    nan = False
    for key in catch:
        correct_answer = catch[key]
        try:
            p_response = int(p_df[key].unique()[0])
            if correct_answer == p_response:
                num_correct += 1
        except:
            nan = True
    nans.append(nan)
            
    if num_correct < 8:
        exclude.append(participant)
    
len(exclude), len(exclude)/len(participant_ids), len([b for b in nans if b == True])

(32, 0.64, 32)

In [50]:
with open('exclude.npy', 'wb') as f:
    np.save(f, np.array(exclude))

In [51]:
for index, row in df.iterrows():
    participant = row['ResponseId']
    if participant in exclude:
        df = df.drop(index)
    


In [52]:
len(df.ResponseId.unique())-2, len(participant_ids) - len(exclude)

(18, 18)

In [53]:
df.to_csv('cleaned_data.csv', index=False)